In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import plotly.io as pio
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os 
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

pio.renderers.default = "iframe_connected"

In [11]:
cities_list = ["Mont Saint Michel","St Malo","Bayeux","Le Havre","Rouen","Paris","Amiens","Lille","Strasbourg","Chateau du Haut Koenigsbourg","Colmar","Eguisheim","Besancon","Dijon","Annecy","Grenoble","Lyon","Gorges du Verdon","Bormes les Mimosas","Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"];

In [12]:
cities = pd.DataFrame(data = cities_list,columns =['city'])
cities["lat"] = 0
cities["lon"] = 0
cities["rain"] = 0
cities["rain_prob"] = 0

In [13]:
# Recovering coordinates for each city
for index,city in cities['city'].iteritems():
    coord = requests.get("https://nominatim.openstreetmap.org/?q=<{},france>&format=json&limit=1".format(city)).json()[0]
    cities.loc[index,"lat"] = coord['lat']
    cities.loc[index,"lon"] = coord['lon']

cities["lat"] = cities['lat'].astype('float')
cities["lon"] = cities['lon'].astype('float')

In [14]:
# Recovering weather data
API_key = '116c6fd579d21074a961ae64785fefbd'
for index,city in cities['city'].iteritems():
    weather_daily = requests.get("https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&exclude=hourly,minutely&appid={}&units=metric".format(cities.loc[index,"lat"],cities.loc[index,"lon"],API_key)).json()['daily']
    daily_rains = [day["pop"]*day["rain"] if 'rain' in day.keys() else 0 for day in weather_daily]
    daily_clouds = [day["clouds"] for day in weather_daily]
    daily_temps = [day["temp"]['max'] for day in weather_daily]
    
    if len(daily_rains) !=0 and len(daily_rains) is not None:
        rain_avg = sum(daily_rains)/len(daily_rains)
    else:
        rain_avg = 0
        
    if len(daily_clouds) !=0 and len(daily_clouds) is not None:
        cloud_avg = sum(daily_clouds)/len(daily_clouds)
    else:
        cloud_avg = 0
    
    if len(daily_temps) !=0 and len(daily_temps) is not None:
        temp_avg = sum(daily_temps)/len(daily_temps)
    else:
        temp_avg = 0
        
    cities.loc[index,"rain"] = rain_avg
    cities.loc[index,"cloudiness"] = cloud_avg
    cities.loc[index,"temperature_max"] = temp_avg
    
    

In [17]:
cities_sorted = cities.sort_values(by='temperature_max',ascending=False,ignore_index=True).loc[:5,:]

In [30]:
# Map : top destinations in France based on temperature from weather data

fig = px.scatter_mapbox(cities_sorted,hover_name = 'city',lat="lat", lon="lon", color="temperature_max",size="rain", zoom=4, mapbox_style="carto-positron")
fig.show()

In [24]:
best_cities = cities_sorted['city'].values.tolist()

['Avignon', 'Nimes', 'Aix en Provence', 'Uzes', 'Carcassonne', 'Toulouse']

In [8]:
# Scraping of booking.com

class bookingSpider(scrapy.Spider):
    
    name ="booking"
    
    start_urls = ["https://www.booking.com/searchresults.fr.html?ss={}".format(city) for city in best_cities]

    def parse(self,response):
        
        hotels = response.xpath("/html/body/div[2]/div[2]/div[2]")
        city = response.request.url.split("ss=")[1].replace("%20",' ')
        
        for hotel in hotels:            
            yield {
                "city" : city,
                "title": hotel.xpath("div/h3/a/text()").getall(),
                "url": hotel.xpath("div/h3/a//@href").getall(),
                "description": hotel.xpath("div/div[1]/p[2]/text()").getall(),
                "scores": hotel.xpath("div/div[1]/div[1]/b").getall()
                  }

filename = "booking.json"

if filename in os.listdir():
        os.remove(filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        filename : {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(bookingSpider)
process.start()

2021-09-11 18:28:59 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-09-11 18:28:59 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Windows-10-10.0.19041-SP0
2021-09-11 18:28:59 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2021-09-11 18:28:59 [scrapy.extensions.telnet] INFO: Telnet Password: 065e6452a5717f7a
2021-09-11 18:28:59 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-09-11 18:28:59 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.

In [2]:
import pandas as pd
# Import file in a dataframe
df =pd.read_json('booking.json')

# Isolating description
descr = df['description']
df2=df.drop(columns='description')

# Description processing
descr_unested = descr.explode()
descr_clean = descr_unested[descr_unested !='\n']

# Data unnest
df_clean = df2.set_index('city').apply(pd.Series.explode).reset_index()

# Concatenation
descr_clean = descr_clean.reset_index(drop=True)
df_final = pd.concat([df_clean,descr_clean],axis=1)

# Data cleaning

# URL
url_add = pd.Series(["https://www.booking.com" for i in range(df_final.shape[0])])

df_final["url"] = url_add.str.cat(df_final["url"])
df_final["url"] = df_final["url"].str.split("?").apply(lambda x: x[0])

# Recovering scores
df_final["scores"] = df_final["scores"].str.split('alt="').apply(lambda x: x[1].split('/')[0] if len(x) ==2 else 0)

# Description
df_final["description"] = df_final["description"].str.replace("\n","")

 Scraping of hotel coordinates : it's necessary to restart the kernel to launch the crawler a second time, then execute the cell just above to load data from previous scraping


In [3]:
import os 

import logging

import scrapy
from scrapy.crawler import CrawlerProcess

class booking_hotelSpider(scrapy.Spider):
    
    name ="hotel_page"
    
    start_urls = [url for url in df_final["url"]]

    def parse(self,response):
          
        yield {
                "coords": response.xpath("/html/body/div[2]/div[2]/div[1]/div[2]/p/a//@data-coords").getall(),
                "scores": response.xpath("/html/body/div[2]/div[2]/div[2]/div[1]/text()").getall()            
                  }

filename = "hotel_page.json"

if filename in os.listdir():
        os.remove(filename)

# Declare a new CrawlerProcess with some settings
process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        filename : {"format": "json"},
    }
})

# Start the crawling using the spider you defined above
process.crawl(booking_hotelSpider)
process.start()

2021-09-11 19:38:19 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2021-09-11 19:38:19 [scrapy.utils.log] INFO: Versions: lxml 4.6.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.1.1, Platform Windows-10-10.0.19041-SP0
2021-09-11 19:38:19 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2021-09-11 19:38:19 [scrapy.extensions.telnet] INFO: Telnet Password: 8845ef846bbd172b
2021-09-11 19:38:19 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2021-09-11 19:38:19 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.

In [3]:
coords =pd.read_json('hotel_page.json')
df_final['lon'] = coords['coords'].apply(lambda x: x[0].split(",")[0]).astype('float')
df_final['lat'] = coords['coords'].apply(lambda x: x[0].split(",")[1]).astype('float')
df_final['scores'] = df_final['scores'].astype('float')

In [36]:
df_final = df_final.dropna()

In [51]:
coords =pd.read_json('hotel_page.json')
coords["scores"]

0       [\nNote sur 878 commentaires   ,  7.4\n]
1      [\nNote sur 1264 commentaires   ,  7.6\n]
2       [\nNote sur 801 commentaires   ,  6.8\n]
3      [\nNote sur 1999 commentaires   ,  8.3\n]
4       [\nNote sur 534 commentaires   ,  7.7\n]
                         ...                    
694    [\nNote sur 1894 commentaires   ,  7.9\n]
695    [\nNote sur 2284 commentaires   ,  8.2\n]
696    [\nNote sur 2918 commentaires   ,  7.4\n]
697     [\nNote sur 180 commentaires   ,  9.1\n]
698    [\nNote sur 2292 commentaires   ,  7.7\n]
Name: scores, Length: 699, dtype: object

In [24]:
text =coords["scores"][2][1]
text


' 6.8\n'

In [38]:
import re
score_search = re.match('\d+\.\d', text)


In [53]:
if re.search('\d+\.\d', text).group(0):
    print('ok')

ok


In [58]:
coords["scores_clean"] = coords["scores"].apply(lambda x: re.search('\d+\.\d', x[1]).group(0) if re.search('\d+\.\d', x[1]).group(0) else 0)
coords["scores_clean"]

AttributeError: 'NoneType' object has no attribute 'group'

In [49]:
# Map : hotels around top destinations based on the first map
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

pio.renderers.default = "iframe_connected"

fig = px.scatter_mapbox(df_final,hover_name = 'title',lat="lat",color='scores', lon="lon", zoom=4, mapbox_style="carto-positron")
fig.show()